In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
path = '/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/07. Optimal sites/Datasets/Данные для поисков оптимальных участков/external_data'

In [3]:
bs_df = pd.read_csv(f'{path}/BITMEX_SIGNALS.csv')

In [7]:
bs_df.head()

,datetime,bitmex_price,aax_price
0,2020-08-10 21:42:00.286741,11832.75,11835.75
1,2020-08-10 21:42:00.899705,11832.75,11835.75
2,2020-08-10 21:42:01.136885,11832.75,11836.00
3,2020-08-10 21:42:01.956848,11832.75,11835.75
4,2020-08-10 21:42:06.316766,11832.25,11835.75


In [4]:
im_df = pd.read_csv(f'{path}/IMBALANCES.csv')
im_df.columns = ['datetime','Signal_Imb']

In [5]:
ml_long = pd.read_csv(f'{path}/pred_long_2020-08-01.csv')
ml_short = pd.read_csv(f'{path}/pred_short_2020-08-01.csv')
ml_long.columns = ['Both','Buy','Sell','datetime']
ml_short.columns = ['Both','Buy','Sell','datetime']

In [6]:
im_df.tail()

,datetime,Signal_Imb
5356776,2020-08-31 23:59:55,Unchanged
5356777,2020-08-31 23:59:56,Unchanged
5356778,2020-08-31 23:59:57,Unchanged
5356779,2020-08-31 23:59:58,Unchanged
5356780,2020-08-31 23:59:59,Unchanged


In [5]:
def funct(model: pd.DataFrame, th = 0.34):
    signal = list()
    for i in range(len(model)):
        signal.append(max(model['Both'][i],model['Buy'][i],model['Sell'][i]))
    
    signals = list()    
    for i in range(len(model)):
        if (signal[i] == model['Both'].values[i]) & (signal[i]>th):
            signals.append(0)
        elif (signal[i] == model['Buy'].values[i]) & (signal[i]>th):
            signals.append(1)
        elif (signal[i] == model['Sell'].values[i]) & (signal[i]>th):
            signals.append(2)
        else:
            signals.append(-1) 
    return signals

In [6]:
ml_long['Signals_Long'] = funct(ml_long, 0.34)
ml_short['Signals_Short'] = funct(ml_short, 0.34)

In [7]:
def get_prediction(long_result: int, short_result: int):
    buy = 'Buy'
    sell = 'Sell'
    both = 'Both'
    no_trade = 'NoTrade'
    result = {
        (-1, -1): 'no_trade',
        (-1, 0): 'sell',
        (-1, 1): 'no_trade',
        (-1, 2): '(sell, True)',
        (0, -1): 'buy',
        (0, 0): 'both',
        (0, 1): 'buy',
        (0, 2): 'no_trade',
        (1, -1): 'buy',
        (1, 0): 'no_trade',
        (1, 1): 'buy',
        (1, 2): 'no_trade',
        (2, -1): 'no_trade',
        (2, 0): 'sell',
        (2, 1): 'no_trade',
        (2, 2): 'sell'
    }.get((long_result, short_result), (no_trade, False))
    return result

In [8]:
df_ml = ml_long[['datetime','Signals_Long']]
df_ml['Signals_Short'] = ml_short['Signals_Short']

In [9]:
signals = list()
for i in tqdm(range(len(df_ml))):
    signals.append(get_prediction(df_ml['Signals_Long'].values[i], df_ml['Signals_Short'].values[i]))
df_ml['Signal_Ml'] = signals

100%|██████████| 108524/108524 [00:00<00:00, 167798.26it/s]


In [10]:
df_ml = df_ml[['datetime','Signal_Ml']]

In [11]:
def get_signals(prices: (float, float)):
    bitmex_price, aax_price = prices
    price_diff = abs(bitmex_price - aax_price)
    price_thr = aax_price * 0.05 / 100
    if (abs(price_diff) <= price_thr):
        return 'Unchanged'
    
    if bitmex_price - aax_price > 0:
        return 'Up'
    else:
        return 'Down'

In [12]:
bitmex_prices = bs_df['bitmex_price'].values
aax_prices = bs_df['aax_price'].values
signals = list(map(get_signals, list(zip(bitmex_prices, aax_prices))))

In [13]:
bs_df['Signal_RP'] = signals

In [14]:
bs_df = bs_df[(bs_df['datetime']>="2020-08-10 21:42:00") & (bs_df['datetime']<"2020-09-01")]
im_df = im_df[(im_df['datetime']>="2020-08-10 21:42:00") & (im_df['datetime']<"2020-09-01")]
df_ml = df_ml[(df_ml['datetime']>="2020-08-10 21:41:40.014954") & (df_ml['datetime']<"2020-09-01")]

In [15]:
bs_df.head()

,datetime,bitmex_price,aax_price,Signal_RP
0,2020-08-10 21:42:00.286741,11832.75,11835.75,Unchanged
1,2020-08-10 21:42:00.899705,11832.75,11835.75,Unchanged
2,2020-08-10 21:42:01.136885,11832.75,11836.00,Unchanged
3,2020-08-10 21:42:01.956848,11832.75,11835.75,Unchanged
4,2020-08-10 21:42:06.316766,11832.25,11835.75,Unchanged


# Основной блок

In [16]:
bs_df['diff_price'] = bs_df['bitmex_price'] - bs_df['aax_price']

In [17]:
df_test = pd.concat([bs_df,im_df])
df_test = df_test.sort_values(by='datetime')
df_test.index = range(len(df_test))

In [18]:
df_test.head()

,datetime,bitmex_price,aax_price,Signal_RP,diff_price,Signal_Imb
0,2020-08-10 21:42:00,NaN,NaN,NaN,NaN,Unchanged
1,2020-08-10 21:42:00.286741,11832.75,11835.75,Unchanged,-3.00,NaN
2,2020-08-10 21:42:00.899705,11832.75,11835.75,Unchanged,-3.00,NaN
3,2020-08-10 21:42:01,NaN,NaN,NaN,NaN,Down
4,2020-08-10 21:42:01.136885,11832.75,11836.00,Unchanged,-3.25,NaN


In [19]:
df_test.iloc[1] = ['2020-08-10 21:42:00.286741', '11832.75','11835.75','Unchanged','-3.00','Unchanged']
df_test = df_test.iloc[1:]
df_test.index = range(len(df_test))

In [22]:
## Для имбалансов
df_test['Signal_Imb'] = df_test['Signal_Imb'].fillna('V')
imb = df_test['Signal_Imb'].values
vlad = list()
v = 'v'
for i in range(len(imb)):
    if imb[i] != 'V':
        vlad.append(imb[i])
        v = imb[i]
    elif imb[i] == 'V':
        vlad.append(v)
df_test['Signal_Imb'] = vlad

df_test = df_test.dropna()
df_test = df_test.sort_values(by='datetime')
df_test.index = range(len(df_test))

In [23]:
df_test.head()

,datetime,bitmex_price,aax_price,Signal_RP,diff_price,Signal_Imb
0,2020-08-10 21:42:00.286741,11832.75,11835.75,Unchanged,-3.00,Unchanged
1,2020-08-10 21:42:00.899705,11832.8,11835.8,Unchanged,-3,Unchanged
2,2020-08-10 21:42:01.136885,11832.8,11836,Unchanged,-3.25,Down
3,2020-08-10 21:42:01.956848,11832.8,11835.8,Unchanged,-3,Down
4,2020-08-10 21:42:06.316766,11832.2,11835.8,Unchanged,-3.5,Down


In [26]:
# Вычисление стандартной волатильности
def calculation_volatility_STD(df_data_close_price:pd.Series,
                               win_size):
    volatility_STD = df_data_close_price.pct_change().rolling(win_size).std()*1_000
    return volatility_STD

In [28]:
df_test['bitmex_price'] = pd.to_numeric(df_test['bitmex_price'])
df_test['aax_price'] = pd.to_numeric(df_test['aax_price'])

In [31]:
df_test['volatility'] = calculation_volatility_STD(df_test['bitmex_price'], 5)
df_test = df_test.dropna()
df_test.index = range(len(df_test))

In [36]:
## Сигналы по волатильности
signals = list()
for i in range(len(df_test)):
    if df_test['volatility'].values[i] < 6.529078e-02:
        signals.append('Low')
    else:
        signals.append('Unchanged')
df_test['Signal_Vol'] = signals

In [38]:
df_ml.head()

,datetime,Signal_Ml
40327,2020-08-10 21:41:40.014954,no_trade
40328,2020-08-10 21:42:03.850807,no_trade
40329,2020-08-10 21:43:48.229892,buy
40330,2020-08-10 21:44:31.596609,sell
40331,2020-08-10 21:44:53.680998,no_trade


In [39]:
start_date = max([df_test['datetime'].iloc[0], df_ml['datetime'].iloc[0]])
finish_date = min([df_test['datetime'].iloc[-1], df_ml['datetime'].iloc[-1]])

In [43]:
df_test = df_test[
    (df_test['datetime'] >= start_date) & 
    (df_test['datetime'] <= finish_date)
]
df_ml = df_ml[
    (df_ml['datetime'] >= start_date) & 
    (df_ml['datetime'] <= finish_date)
]

In [45]:
df_ml['datetime'] = pd.to_datetime(df_ml['datetime'])
df_ml.index = range(len(df_ml))

In [46]:
df_ml.head(2)

,datetime,Signal_Ml
0,2020-08-10 21:43:48.229892,buy
1,2020-08-10 21:44:31.596609,sell


In [52]:
mls = df_ml['Signal_Ml'].values 
ml_datetimes = df_ml['datetime'].values

In [53]:
value = 'Unknown'
values = []
for i, ml in enumerate(mls):    
    if value != ml:      
        value = ml
        values.append((value, i))        

In [58]:
items = []
for i in tqdm(range(len(values) - 1)):
    d = {}
    first_date = ml_datetimes[values[i][1]]
    second_date = ml_datetimes[values[i + 1][1]]
    
    temp_df = df_test[(df_test['datetime'] >= first_date) & (df_test['datetime'] < second_date)]
    
    if len(temp_df) == 0:
        continue
        
    first_bp = temp_df['bitmex_price'].iloc[0]
    last_bp = temp_df['bitmex_price'].iloc[-1]
    max_bp = temp_df['bitmex_price'].max()
    min_bp = temp_df['bitmex_price'].min()
    
    first_ap = temp_df['aax_price'].iloc[0]
    last_ap = temp_df['aax_price'].iloc[-1]
    max_ap = temp_df['aax_price'].max()
    min_ap = temp_df['aax_price'].min()
    
    first_signal_rp = temp_df['Signal_RP'].iloc[0]
    last_signal_rp = temp_df['Signal_RP'].iloc[-1]
    
    first_signal_imb = temp_df['Signal_Imb'].iloc[0]
    last_signal_imb = temp_df['Signal_Imb'].iloc[-1]
    
    first_signal_vol = temp_df['Signal_Vol'].iloc[0]
    last_signal_vol = temp_df['Signal_Vol'].iloc[-1]
    
    mlsignals = values[i][0]
    
    d['first_date'] = first_date
    d['second_date'] = second_date
    
    d['first_bp'] = first_bp
    d['last_bp'] = last_bp
    d['max_bp'] = max_bp
    d['min_bp'] = min_bp
    
    d['first_ap'] = first_ap
    d['last_ap'] = last_ap
    d['max_ap'] = max_ap
    d['min_ap'] = min_ap
    
    d['first_signal_RP'] = first_signal_rp
    d['last_signal_RP'] = last_signal_rp
    
    d['first_signal_Imb'] = first_signal_imb
    d['last_signal_Imb'] = last_signal_imb
    
    d['first_signal_Vol'] = first_signal_vol
    d['last_signal_Vol'] = last_signal_vol
    
    d['ML_signals'] = mlsignals
    
    items.append(d)

100%|██████████| 26611/26611 [01:39<00:00, 266.87it/s]


In [59]:
df = pd.DataFrame(items)

In [60]:
df['first_date'] = pd.to_datetime(df['first_date'])
df['second_date'] = pd.to_datetime(df['second_date'])
df['date_diff'] = df['second_date'] - df['first_date']
df['price_diff'] = df['first_ap'] - df['last_ap']

In [61]:
def price_move(diff_price: float):
    if abs(diff_price) <= 1:
        return 'Unchanged'
    
    if diff_price > 0:
        return 'Down'
    else:
        return 'Up'

In [62]:
price_diffs = df['price_diff'].values
df['price_move'] = list(map(price_move, price_diffs))

In [63]:
df.head(3)

,first_date,second_date,first_bp,last_bp,max_bp,min_bp,first_ap,last_ap,max_ap,min_ap,first_signal_RP,last_signal_RP,first_signal_Imb,last_signal_Imb,first_signal_Vol,last_signal_Vol,ML_signals,date_diff,price_diff,price_move
0,2020-08-10 21:43:48.229892,2020-08-10 21:44:31.596609,11832.25,11834.25,11834.25,11832.25,11831.25,11831.25,11831.5,11831.25,Unchanged,Unchanged,Unchanged,Unchanged,Low,Low,buy,00:00:43.366717,0.0,Unchanged
1,2020-08-10 21:44:31.596609,2020-08-10 21:44:53.680998,11834.25,11833.25,11834.25,11833.25,11831.50,11831.50,11831.5,11831.50,Unchanged,Unchanged,Down,Down,Low,Low,sell,00:00:22.084389,0.0,Unchanged
2,2020-08-10 21:44:53.680998,2020-08-10 21:45:21.707390,11832.75,11817.25,11832.75,11817.25,11831.50,11831.50,11831.5,11831.50,Unchanged,Down,Down,Unchanged,Low,Low,no_trade,00:00:28.026392,0.0,Unchanged


In [64]:
df[df['price_move']!='Unchanged']

,first_date,second_date,first_bp,last_bp,max_bp,min_bp,first_ap,last_ap,max_ap,min_ap,first_signal_RP,last_signal_RP,first_signal_Imb,last_signal_Imb,first_signal_Vol,last_signal_Vol,ML_signals,date_diff,price_diff,price_move
3,2020-08-10 21:45:21.707390,2020-08-10 21:45:55.165998,11818.75,11822.50,11822.50,11818.75,11831.50,11819.25,11831.50,11819.00,Down,Unchanged,Unchanged,Unchanged,Unchanged,Low,buy,00:00:33.458608,12.25,Down
4,2020-08-10 21:45:55.165998,2020-08-10 21:49:01.744957,11822.75,11832.25,11832.25,11822.75,11819.25,11828.00,11830.50,11819.00,Unchanged,Unchanged,Unchanged,Unchanged,Low,Low,sell,00:03:06.578959,-8.75,Up
7,2020-08-10 21:50:10.773137,2020-08-10 21:50:55.072320,11836.25,11838.25,11838.25,11836.25,11828.75,11834.75,11834.75,11828.50,Up,Unchanged,Unchanged,Up,Low,Low,sell,00:00:44.299183,-6.00,Up
8,2020-08-10 21:50:55.072320,2020-08-10 21:51:48.609393,11838.50,11839.75,11839.75,11838.50,11834.75,11840.00,11840.00,11834.75,Unchanged,Unchanged,Up,Up,Low,Low,no_trade,00:00:53.537073,-5.25,Up
9,2020-08-10 21:51:48.609393,2020-08-10 21:53:20.298742,11839.75,11844.25,11848.25,11839.75,11840.25,11846.00,11846.25,11840.25,Unchanged,Unchanged,Up,Unchanged,Low,Low,sell,00:01:31.689349,-5.75,Up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25380,2020-08-31 23:35:36.734740,2020-08-31 23:42:49.534342,11689.75,11680.25,11689.75,11680.25,11697.75,11691.50,11697.75,11681.25,Down,Down,Unchanged,Unchanged,Low,Low,sell,00:07:12.799602,6.25,Down
25381,2020-08-31 23:42:49.534342,2020-08-31 23:47:30.707062,11682.50,11693.75,11693.75,11682.50,11691.50,11697.50,11702.50,11689.75,Down,Unchanged,Unchanged,Unchanged,Unchanged,Low,buy,00:04:41.172720,-6.00,Up
25382,2020-08-31 23:47:30.707062,2020-08-31 23:52:43.013600,11693.75,11672.75,11693.75,11672.75,11697.75,11679.25,11698.50,11679.25,Unchanged,Down,Unchanged,Unchanged,Low,Low,sell,00:05:12.306538,18.50,Down
25383,2020-08-31 23:52:43.013600,2020-08-31 23:53:11.071501,11666.50,11660.25,11666.50,11660.25,11679.25,11672.00,11679.50,11667.75,Down,Down,Unchanged,Unchanged,Unchanged,Low,no_trade,00:00:28.057901,7.25,Down


In [110]:
# m_time = list()
# for i in range(1, len(df_ml)):
#     m_time.append(df_ml['datetime'][i]-df_ml['datetime'][i-1])
# max(m_time)

Timedelta('0 days 00:08:07.690352')

In [115]:
np.array(m_time).mean()

Timedelta('0 days 00:00:26.725726')

In [96]:
df[df['price_move']=='Down']['ML_signals'].value_counts(normalize=True)*100

sell        37.619361
no_trade    32.713204
buy         21.122819
both         8.544616
Name: ML_signals, dtype: float64

In [97]:
df[df['price_move']=='Up']['ML_signals'].value_counts(normalize=True)*100

sell        32.322900
buy         31.532125
no_trade    26.161450
both         9.983526
Name: ML_signals, dtype: float64

In [98]:
df[df['price_move']=='Unchanged']['ML_signals'].value_counts(normalize=True)*100

no_trade    34.040625
sell        32.220796
buy         25.515367
both         8.223212
Name: ML_signals, dtype: float64

In [100]:
df[df['price_move']=='Down'][['first_bp','last_bp','first_ap','last_ap','first_signal_RP','last_signal_RP','ML_signals']]

,first_bp,last_bp,first_ap,last_ap,first_signal_RP,last_signal_RP,ML_signals
3,11818.75,11822.50,11831.50,11819.25,Down,Unchanged,buy
10,11844.25,11836.75,11846.25,11836.75,Unchanged,Unchanged,no_trade
15,11847.25,11841.25,11851.00,11847.25,Unchanged,Down,no_trade
17,11837.25,11837.25,11846.75,11840.25,Down,Unchanged,no_trade
19,11836.25,11830.00,11840.25,11834.00,Unchanged,Unchanged,no_trade
...,...,...,...,...,...,...,...
25379,11689.75,11689.75,11699.00,11697.50,Down,Down,buy
25380,11689.75,11680.25,11697.75,11691.50,Down,Down,sell
25382,11693.75,11672.75,11697.75,11679.25,Unchanged,Down,sell
25383,11666.50,11660.25,11679.25,11672.00,Down,Down,no_trade


In [82]:
len(df[df['price_move']=='Down'])

6074

In [83]:
df_vlad = df[(df['price_move']=='Down')]

In [88]:
len(df_vlad[(df_vlad['first_signal_RP']!="Up") |
            (df_vlad['last_signal_RP']!="Up")])

len(df_vlad[(df_vlad['first_signal_RP']=="Unchanged") &
            (df_vlad['last_signal_RP']=="Unchanged")])

6046

2863